In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Conv2D, Conv1D, Input, Reshape, Dense, Flatten
from keras.layers import BatchNormalization, Activation, AveragePooling1D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [2]:
xtrain = np.load('./Data/train_features.npy')
ytrain = np.load('./Data/train_labels.npy')

xval = np.load('./Data/val_features.npy')
yval = np.load('./Data/val_labels.npy')

In [58]:
#### Model ####

X = Input(batch_shape = (None, 60, 41, 2))

conv1 = Conv2D(128, (3, 41), strides = (1,41), padding = 'same', kernel_initializer = 'he_normal')(X)
bn1 = BatchNormalization()(conv1)
act1 = Activation('relu')(bn1)

reshape = Reshape((60,128))(act1)

conv2 = Conv1D(256, 3, strides = 1, padding = 'same', kernel_initializer = 'he_normal')(reshape)
bn2 = BatchNormalization()(conv2)
act2 = Activation('relu')(bn2)

pool = AveragePooling1D(pool_size = act2.get_shape().as_list()[1], 
                        strides=1, padding='valid')(act2)

flat = Flatten()(pool)
d = Dense(4, kernel_initializer='he_normal')(flat)
bn_d = BatchNormalization()(d)
Yhat = Activation('softmax')(bn_d)

model = Model(X, Yhat)

#### Training Set Up ####

weight_dict = {0: float(ytrain[ytrain[:,0]==1].shape[0])/ytrain[ytrain[:,0]==1].shape[0],
               1: float(ytrain[ytrain[:,0]==1].shape[0])/ytrain[ytrain[:,1]==1].shape[0],
               2: float(ytrain[ytrain[:,0]==1].shape[0])/ytrain[ytrain[:,2]==1].shape[0],
               3: float(ytrain[ytrain[:,0]==1].shape[0])/ytrain[ytrain[:,3]==1].shape[0]}

lr = 1e-6                                                                                                                                                           
dcy = 0.0
m = 0.0                                                                                                                                                                           
sgd = SGD(lr=lr, momentum=m, decay=dcy,  nesterov=True)

name = './history/model_lr%.1ef_dcy%.2f_m%.2f' % (lr, dcy, m)
checkpointer = ModelCheckpoint(filepath = name + '_{epoch:03d}-{val_acc:.2f}.h5',
                               monitor = 'val_acc', verbose=0, save_best_only=True)

model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])




In [59]:
model.fit(xtrain, ytrain, batch_size=64, epochs=6, verbose=1, callbacks = [checkpointer], 
          validation_data= (xval, yval), class_weight=None)

Train on 8354 samples, validate on 1660 samples
Epoch 1/6
8354/8354 [==============================] - 22s - loss: 1.4664 - acc: 0.3996 - val_loss: 1.5573 - val_acc: 0.3199
Epoch 2/6
8354/8354 [==============================] - 21s - loss: 1.3273 - acc: 0.4587 - val_loss: 1.7561 - val_acc: 0.2867
Epoch 3/6
8354/8354 [==============================] - 21s - loss: 1.1875 - acc: 0.5306 - val_loss: 1.9345 - val_acc: 0.3428
Epoch 4/6
8354/8354 [==============================] - 21s - loss: 1.0867 - acc: 0.5758 - val_loss: 2.2220 - val_acc: 0.2711
Epoch 5/6
8354/8354 [==============================] - 21s - loss: 1.0416 - acc: 0.6069 - val_loss: 2.5870 - val_acc: 0.2024
Epoch 6/6
8354/8354 [==============================] - 21s - loss: 1.0156 - acc: 0.6307 - val_loss: 2.8287 - val_acc: 0.1512


In [70]:
yval[yval[:,0]==1].shape[0]/float(yval.shape[0])

0.3921686746987952